In [132]:
import os
import re
import pandas as pd

# caminho para a pasta onde estão os arquivos CSV
caminho_pasta = 'dados'
arquivo = 'Mortalidade_Geral_2010.csv'

# Ler o arquivo CSV com o pandas
df = pd.read_csv(os.path.join(caminho_pasta, arquivo), delimiter=';', encoding='ISO-8859-1')
# Criar coluna ano_arquivo para identificar arquivo fonte
df['ano_arquivo'] = re.findall(r'\d+', arquivo)[0]
# Criar unicidade para não repetir o dado em futuras atualizações
df['pk_base_origem'] = df['ano_arquivo'] + '_' + df['CONTADOR'].astype(str)
# Realizar correções nas datas de nascimento e óbito
df['DTOBITO'] = pd.to_datetime(df['DTOBITO'], format='%d%m%Y', errors='coerce')
df['DTNASC'] = pd.to_datetime(df['DTNASC'], format='%d%m%Y', errors='coerce')
# Criando a coluna 'ano_obito'
df['ano_obito'] = df['DTOBITO'].dt.year.astype(float).astype(pd.Int64Dtype()).astype(str).where(df['DTOBITO'].notna())
# Criando a coluna 'quadrimestre_obito' usando pd.cut()
df['quadrimestre_obito'] = pd.cut(df['DTOBITO'].dt.month, bins=[1, 5, 9, 13], labels=[1, 2, 3], right=False)

# Função para transformar a idade conforme as regras especificadas pela fonte
def transformar_idade(idade):
    if pd.isnull(idade):
        return None
    elif str(idade).startswith('4'):
        idade_str = str(idade).split('.')[0][1:]
        if idade_str:
            return int(idade_str)
    return 0
# Aplicando a transformação na coluna de idade
df['IDADE'] = df['IDADE'].apply(transformar_idade)

# Transfromar tipo de óbito
mapeamento = {1: 'Fetal', 2: 'Não Fetal', 3: 'Ignorado'}
df['TIPOBITO'] = df['TIPOBITO'].replace(mapeamento)
# Extrair os 6 primeiros dígitos da coluna CODMUNRES
df['CODMUNRES'] = df['CODMUNRES'].astype(str).str.slice(stop=6)
# Renomear colunas
df = df.rename(columns={
    'DTOBITO': 'dt_obito',
    'DTNASC':'dt_nascimento',
    'IDADE':'idade',
    'TIPOBITO':'tp_obito',
    'CAUSABAS':'cid10',
    'CODMUNRES':'cd_municipio_residencia'
    })
# Selecionar coluna desejadas
df = df[[
    "pk_base_origem","ano_arquivo","ano_obito","quadrimestre_obito","dt_obito",
    "dt_nascimento","idade","tp_obito","cid10","cd_municipio_residencia"
    ]]

C:\Users\marco\AppData\Local\Temp\ipykernel_8412\4131358260.py:10: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(caminho_pasta, arquivo), delimiter=';', encoding='ISO-8859-1')


In [133]:
# exibe o dataframe final
df.head()

,pk_base_origem,ano_arquivo,ano_obito,quadrimestre_obito,dt_obito,dt_nascimento,idade,tp_obito,cid10,cd_municipio_residencia
0,2010_1,2010,2010,2,2010-08-06,1945-04-09,65.0,Não Fetal,R98,120040
1,2010_2,2010,2010,2,2010-08-06,1912-01-20,98.0,Não Fetal,I219,120040
2,2010_3,2010,2010,3,2010-10-02,2010-03-17,0.0,Não Fetal,R98,120040
3,2010_4,2010,2010,1,2010-04-07,2010-04-07,0.0,Não Fetal,P969,120042
4,2010_5,2010,2010,2,2010-05-13,1971-08-04,38.0,Não Fetal,X999,120020


In [134]:
df.dtypes

pk_base_origem                     object
ano_arquivo                        object
ano_obito                          object
quadrimestre_obito               category
dt_obito                   datetime64[ns]
dt_nascimento              datetime64[ns]
idade                             float64
tp_obito                           object
cid10                              object
cd_municipio_residencia            object
dtype: object

In [135]:
df.columns

Index(['pk_base_origem', 'ano_arquivo', 'ano_obito', 'quadrimestre_obito',
       'dt_obito', 'dt_nascimento', 'idade', 'tp_obito', 'cid10',
       'cd_municipio_residencia'],
      dtype='object')

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136947 entries, 0 to 1136946
Data columns (total 10 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   pk_base_origem           1136947 non-null  object        
 1   ano_arquivo              1136947 non-null  object        
 2   ano_obito                1130681 non-null  object        
 3   quadrimestre_obito       1130681 non-null  category      
 4   dt_obito                 1130681 non-null  datetime64[ns]
 5   dt_nascimento            1124294 non-null  datetime64[ns]
 6   idade                    1136943 non-null  float64       
 7   tp_obito                 1136947 non-null  object        
 8   cid10                    1136947 non-null  object        
 9   cd_municipio_residencia  1136947 non-null  object        
dtypes: category(1), datetime64[ns](2), float64(1), object(6)
memory usage: 79.2+ MB


In [137]:
df.shape

(1136947, 10)

In [138]:
df.isna().sum()

pk_base_origem                 0
ano_arquivo                    0
ano_obito                   6266
quadrimestre_obito          6266
dt_obito                    6266
dt_nascimento              12653
idade                          4
tp_obito                       0
cid10                          0
cd_municipio_residencia        0
dtype: int64